In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('WDIData.csv')

In [4]:
df.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       'Unnamed: 67'],
      dtype='object')

In [5]:
df = df[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '2020']]

In [6]:
df = df.dropna(subset=['2020'])

In [7]:
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,2020
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,19.501837
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,7.599289
2,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,38.793983
3,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,45.609604
4,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,30.163364


In [11]:
df = df.pivot(index='Country Name', columns='Indicator Name', values='2020')

In [12]:
df

Indicator Name,ARI treatment (% of children under 5 taken to a health provider),Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",Adjusted net national income (annual % growth),Adjusted net national income (constant 2015 US$),Adjusted net national income (current US$),...,Women who believe a husband is justified in beating his wife (any of five reasons) (%),Women who believe a husband is justified in beating his wife when she argues with him (%),Women who believe a husband is justified in beating his wife when she burns the food (%),Women who believe a husband is justified in beating his wife when she goes out without telling him (%),Women who believe a husband is justified in beating his wife when she neglects the children (%),Women who believe a husband is justified in beating his wife when she refuses sex with him (%),Women who were first married by age 15 (% of women ages 20-24),Women who were first married by age 18 (% of women ages 20-24),Women's share of population ages 15+ living with HIV (%),Young people (ages 15-24) newly infected with HIV
Country Name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,NaN,33.200000,15.900000,82.600000,97.699997,97.066711,99.500000,NaN,NaN,1.858582e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.100000,500.0
Africa Eastern and Southern,NaN,19.501837,7.599289,38.793983,45.609604,30.163364,73.775675,-4.999989,8.114053e+11,7.625661e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.310464,NaN
Africa Western and Central,NaN,14.903839,2.959758,30.873764,52.082053,25.459506,83.165402,5.794022,6.928618e+11,6.522694e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.542146,NaN
Albania,NaN,81.300000,64.600000,92.900000,100.000000,100.000000,100.000000,NaN,NaN,1.226674e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.400000,100.0
Algeria,NaN,99.700000,98.800000,99.900000,99.804131,99.579903,99.884010,-7.566353,1.393817e+11,1.184676e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.800000,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Bank and Gaza,77.3,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.7,13.4,NaN,NaN
World,NaN,69.570111,48.459512,86.391980,90.435577,82.544380,97.235240,-4.262697,6.590557e+13,6.858044e+13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.800000,430000.0
"Yemen, Rep.",NaN,61.500000,41.700000,93.300000,73.757927,61.941116,93.113457,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.400000,200.0


In [8]:
df['Indicator Name'].unique()

array(['Access to clean fuels and technologies for cooking (% of population)',
       'Access to clean fuels and technologies for cooking, rural (% of rural population)',
       'Access to clean fuels and technologies for cooking, urban (% of urban population)',
       ...,
       'Proportion of population pushed below the $1.90 ($ 2011 PPP) poverty line by out-of-pocket health care expenditure (%)',
       'Proportion of population pushed below the $3.20 ($ 2011 PPP) poverty line by out-of-pocket health care expenditure (%)',
       'Consumption of iodized salt (% of households)'], dtype=object)

In [13]:
df.to_parquet('WDIData2020.parquet')

In [19]:
pd.read_parquet('WDIData2020.parquet').head()

Indicator Name,ARI treatment (% of children under 5 taken to a health provider),Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",Adjusted net national income (annual % growth),Adjusted net national income (constant 2015 US$),Adjusted net national income (current US$),...,Women who believe a husband is justified in beating his wife (any of five reasons) (%),Women who believe a husband is justified in beating his wife when she argues with him (%),Women who believe a husband is justified in beating his wife when she burns the food (%),Women who believe a husband is justified in beating his wife when she goes out without telling him (%),Women who believe a husband is justified in beating his wife when she neglects the children (%),Women who believe a husband is justified in beating his wife when she refuses sex with him (%),Women who were first married by age 15 (% of women ages 20-24),Women who were first married by age 18 (% of women ages 20-24),Women's share of population ages 15+ living with HIV (%),Young people (ages 15-24) newly infected with HIV
Country Name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,NaN,33.200000,15.900000,82.600000,97.699997,97.066711,99.500000,NaN,NaN,1.858582e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.100000,500.0
Africa Eastern and Southern,NaN,19.501837,7.599289,38.793983,45.609604,30.163364,73.775675,-4.999989,8.114053e+11,7.625661e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.310464,NaN
Africa Western and Central,NaN,14.903839,2.959758,30.873764,52.082053,25.459506,83.165402,5.794022,6.928618e+11,6.522694e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.542146,NaN
Albania,NaN,81.300000,64.600000,92.900000,100.000000,100.000000,100.000000,NaN,NaN,1.226674e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.400000,100.0
Algeria,NaN,99.700000,98.800000,99.900000,99.804131,99.579903,99.884010,-7.566353,1.393817e+11,1.184676e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.800000,200.0


In [ ]:
df.to_parquet()

In [3]:
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,17.196986,17.597176,18.034249,18.345878,18.695306,19.149942,19.501837,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.580066,6.786218,6.941323,7.096843,7.254828,7.460783,7.599289,NaN,NaN,NaN
2,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,37.857526,38.204173,38.303515,38.421813,38.482409,38.692053,38.793983,NaN,NaN,NaN
3,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,31.824950,33.744405,38.733352,40.092163,42.880977,44.073912,45.609604,NaN,NaN,NaN
4,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,17.485006,16.329765,24.372504,25.153292,27.227391,29.383000,30.163364,NaN,NaN,NaN
